# AWS Machine Learning 목적별 가속기 튜토리얼
## [AWS Trainium](https://aws.amazon.com/machine-learning/trainium/)과 [AWS Inferentia](https://aws.amazon.com/machine-learning/inferentia/)를 [Amazon SageMaker](https://aws.amazon.com/sagemaker/)와 함께 사용하여 ML 워크로드를 최적화하는 방법 알아보기
## 파트 3/3 - Bert 모델을 AWS Inferentia 2로 컴파일하고 SageMaker + [Hugging Face Optimum Neuron](https://huggingface.co/docs/optimum-neuron/index)으로 배포하기

**SageMaker studio 커널: PyTorch 1.13 Python 3.9 CPU - ml.t3.medium** 

이 튜토리얼에서는 모델을 AWS Inferentia로 컴파일한 다음 AWS Inferentia2로 구동되는 SageMaker 실시간 엔드포인트에 배포하는 방법을 배우게 됩니다. 먼저 모델을 컴파일하기 위해 SageMaker 작업을 시작하겠습니다. 이 작업은 한 번만 수행하면 됩니다. 그 후에는 모델을 SageMaker 엔드포인트에 배포하고 마지막으로 예측 결과를 얻을 수 있습니다.

섹션 02에서는 Optimum Neuron API에서 메타데이터를 추출하고 현재 테스트/지원되는 모델이 포함된 테이블을 렌더링합니다(목록에 없는 유사한 모델도 호환될 수 있지만 직접 확인해야 합니다). 이 테이블은 배포에 선택할 수 있는 모델을 이해하는 데 중요합니다. 그러나 모델을 미세 조정해야 하는 경우, **파트 2** 노트북에서 유사한 테이블을 확인하여 HF Optimum Neuron을 사용하여 AWS Trainium으로 미세 조정할 수 있는 모델을 확인하세요. 이렇게 하면 엔드투엔드 솔루션을 계획하고 지금 바로 구현을 시작할 수 있습니다.

## 1) 필요한 패키지 설치

In [1]:
%pip install -r requirements.txt

In [2]:
%pip install -U sagemaker

## 2) 지원되는 모델/작업

이름 뒤에 **[TP]**가 있는 모델은 텐서 병렬 처리를 지원합니다

In [3]:
from IPython.display import Markdown, display

display(Markdown("../docs/optimum_neuron_models.md"))

## 3) 사전 훈련된 모델을 AWS Inferentia로 컴파일하기

**중요:** 이전 노트북 **02_ModelFineTuning** 또는 AWS Console/SageMaker에서 **SageMaker 훈련 작업 이름**을 복사하여 **training_job_name** 변수를 설정하세요. 미세 조정된 모델을 컴파일 작업의 입력으로 사용할 것이기 때문에 필요합니다.

In [4]:
import os
import boto3
import sagemaker

print(sagemaker.__version__)
if not sagemaker.__version__ >= "2.146.0": print("You need to upgrade or restart the kernel if you already upgraded")

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = sess.boto_region_name

training_job_name=""

if os.path.isfile("training_job_name.txt"): training_job_name = open("training_job_name.txt", "r").read().strip()
assert len(training_job_name)>0, "Please copy the name of the training_job you ran in the previous notebook and set training_job_name"
checkpoint_s3_uri=f"s3://{bucket}/output/{training_job_name}/output/model.tar.gz"

if not os.path.isdir('src'): os.makedirs('src', exist_ok=True)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")
print(f"Training job name: {training_job_name}")
print(f"Model S3 URI: {checkpoint_s3_uri}")

### 3.1) SageMaker에서 호출될 컴파일 스크립트

In [5]:
!pygmentize src/compile.py

In [6]:
!pygmentize src/requirements.txt

### 3.2) SageMaker Estimator
이 객체는 컴파일 작업(SageMaker 훈련 작업)을 구성하는 데 도움이 됩니다.

이 작업은 **compile.py** 스크립트를 호출하여 모델을 Inferentia2로 컴파일한 다음 배포를 위한 아티팩트를 저장합니다.

In [7]:
task="SequenceClassification"
# Source: https://huggingface.co/docs/optimum-neuron/guides/export_model#exporting-a-model-to-neuron-via-neuronmodel
input_shapes={"batch_size": 1, "sequence_length": 512}

In [8]:
import json
import logging
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="compile.py", # Specify your train script
    source_dir="src",
    role=role,
    sagemaker_session=sess,
    container_log_level=logging.DEBUG,
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    output_path=f"s3://{bucket}/output",
    disable_profiler=True,
    
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.18.0-ubuntu20.04",
    
    volume_size = 512,
    hyperparameters={     
        "task": task,
        "input_shapes": f"'{json.dumps(input_shapes)}'",
        "dynamic_batch_size": True
    }
)
estimator.framework_version = '1.13.1' # workround when using image_uri

In [9]:
estimator.fit({"checkpoint": checkpoint_s3_uri})

## 4) SageMaker 실시간 엔드포인트 배포

In [10]:
import logging
from sagemaker.utils import name_from_base
from sagemaker.pytorch.model import PyTorchModel

# depending on the inf2 instance you deploy the model you'll have more or less accelerators
# we'll ask SageMaker to launch 1 worker per core

model_data=estimator.model_data
print(f"Model data: {model_data}")

instance_type_idx=0 # default ml.inf2.xlarge
instance_types=['ml.inf2.xlarge', 'ml.inf2.8xlarge', 'ml.inf2.24xlarge','ml.inf2.48xlarge']
num_workers=[2,2,12,24]

print(f"Instance type: {instance_types[instance_type_idx]}. Num SM workers: {num_workers[instance_type_idx]}")
pytorch_model = PyTorchModel(
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.18.0-ubuntu20.04",
    model_data=model_data,
    role=role,    
    name=name_from_base('bert-spam-classifier'),
    sagemaker_session=sess,
    container_log_level=logging.DEBUG,
    model_server_workers=num_workers[instance_type_idx], # 1 worker per inferentia chip
    framework_version="1.13.1",
    env = {
        'SAGEMAKER_MODEL_SERVER_TIMEOUT': '3600',
        'TASK': task
    }
    # for production it is important to define vpc_config and use a vpc_endpoint
    #vpc_config={
    #    'Subnets': ['<SUBNET1>', '<SUBNET2>'],
    #    'SecurityGroupIds': ['<SECURITYGROUP1>', '<DEFAULTSECURITYGROUP>']
    #}
)
pytorch_model._is_compiled_model = True

In [11]:
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_types[instance_type_idx],
    model_data_download_timeout=3600, # it takes some time to download all the artifacts and load the model
    container_startup_health_check_timeout=1800
)

## 5) 간단한 테스트 실행

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [ ]:
import time

labels={0: "not spam", 1: "spam"}
not_spam=" Deezer.com 10,406,168 Artist DB\n\nWe have scraped the Deezer Artist DB, right now there are 10,406,168 listings according to Deezer.com\n\nPlease note in going through part of the list, it is obvious there are mistakes inside their system.\n\nExamples include and Artist with &amp; in its name might also be found with "and" but the Albums for each have different totals etc. Have no clue if there are duplicate albums etc do this error in their system. Even a comma in a name could mean the Artist shows up more than once, I saw in 1 instance that 1 Artist had 6 different ArtistIDs due to spelling errors.\n\nSo what is this DB, very simple, it gives you the ArtistID and the actual name of the Artist in another column. If you want to see the artist you add the baseurl to the ArtistID\n\nAn example is ArtistID 115 is AC/DC\n\n[https://www.deezer.com/us/artist/115](https://www.deezer.com/us/artist/115)\n\nYou do not have to use [https://www.deezer.com/us/artist/](https://www.deezer.com/us/artist/) if your first language is other than English, just see if Deezer supports your language and use that baseref\n\nFrench for example is [https://www.deezer.com/fr/artist/115](https://www.deezer.com/fr/artist/115)\n\nI am providing the DB in 3 different formats:\n\n \n\nI tried posting download links here but it seems Reddit does not like that so get them here:\n\n[https://pastebin\\[DOT\\]com/V3KJbgif](https://pastebin.com/V3KJbgif)\n\n&amp;#x200B;\n\n**Special thanks go to** [**/user/KoalaBear84**](https://www.reddit.com/user/KoalaBear84) **for writing the scraper.**\n\n&amp;#x200B;\n\n**Cross Posted to related Reddit Groups**"
spam="🚨 ATTENTION ALL USERS! 🚨\n\n🆘 Are you looking for a way to GET RICH QUICK? 🆘\n\n💰 Don't waste your time with boring old jobs! 💰\n\n💸 Join our CRAZY MONEY-MAKING SYSTEM today! 💸\n\n🤑 Just sign up and start earning BIG BUCKS right away! 🤑\n\n👉 Plus, if you refer your friends, you'll get even MORE CASH! 👈\n\n🔥 This is the HOTTEST OFFER of the year! 🔥\n\n👍 Don't wait"

for i,text in enumerate([not_spam, spam]):
    t=time.time()
    pred = predictor.predict({"prompt": text})
    elapsed = (time.time()-t)*1000
    print(f"Elapsed time: {elapsed}")
    print(f"Pred: {i} - {labels[pred[0][0]]} / score: {pred[0][1]}")

## 5) Cleanup

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()